# Clase Práctica #5 (Compilación)

En esta clase estaremos adaptando el evaluador de expresiones aritméticas para trabajar sobre un _AST (Abstact Syntax Tree)_. Recordemos que el AST posee una estructura más cómoda para evaluar las reglas semánticas que el árbol de derivación. Además, evaluar en el AST en lugar que desde las reglas de la gramática atributada, simplifica significativamente la implementación de las reglas semánticas.

## Jerarquía del AST

Definamos una jerarquía de clases para los nodos del _AST_ del lenguaje de expresiones aritméticas. Utilizaremos las clases `Node` y `BinaryNode` como definiciones abstractas para agrupar y compactar la implementación de sus descendientes. Los nodos del AST serán exclusivamente instancias de `ConstantNumberNode`, `PlusNode`, `MinusNode`, `StarNode` y `DivNode`.

In [46]:
class Node:
    def evaluate(self):
        raise NotImplementedError()

class ConstantNumberNode(Node):
    def __init__(self, lex):
        self.lex = lex
        self.value = float(lex)
        
    def evaluate(self):
        return self.value
        

class BinaryNode(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
    def evaluate(self):
        lvalue = self.left.evaluate()
        rvalue = self.right.evaluate()
        return self.operate(lvalue, rvalue)
    
    @staticmethod
    def operate(lvalue, rvalue):
        raise NotImplementedError()
        

class PlusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue + rvalue

class MinusNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue - rvalue

class StarNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue * rvalue

class DivNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue / rvalue


Veamos como luce una instancia concreta de un AST de expresiones. Nótese que la precedencia de los operadores debe seguir atrapada en el AST puesto que solo se desecharon los atributos sintácticos.

In [47]:
from cmp.ast import get_printer
printer = get_printer(AtomicNode=ConstantNumberNode, BinaryNode=BinaryNode)

print(printer(
    PlusNode(
        MinusNode(
            ConstantNumberNode('5'),
            ConstantNumberNode('6')
        ), ConstantNumberNode('9')
    )
))

\__<expr> PlusNode <expr>
	\__<expr> MinusNode <expr>
		\__ ConstantNumberNode: 5
		\__ ConstantNumberNode: 6
	\__ ConstantNumberNode: 9


## Construcción del AST

Pasemos a definir la gramática del lenguaje de expresiones aritméticas junto con las reglas semánticas para formar el _AST_. Las reglas quedarán muy similares a las de la clase anterior, pero esta vez en lugar de operar los valores, construiremos el nodo del AST que denota la operación.

In [48]:
from cmp.pycompiler import Grammar
from cmp.utils import pprint, inspect

G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y = G.NonTerminals('T F X Y')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) num')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
X %= plus + T + X, lambda h, s: s[3], None, None, lambda h,s: PlusNode(h[0], s[2])
X %= minus + T + X, lambda h, s: s[3], None, None, lambda h,s: MinusNode(h[0], s[2])
X %= G.Epsilon, lambda h, s: h[0]
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
T %= F + Y, lambda h, s: s[2], None, lambda h, s: s[1]
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
Y %= star + F + Y, lambda h, s: s[3], None, None, lambda h, s: StarNode(h[0], s[2])
Y %= div + F + Y, lambda h, s: s[3], None, None, lambda h, s: DivNode(h[0], s[2])
Y %= G.Epsilon, lambda h, s: h[0]
#                                                                         #
# ======================= { F --> num | ( E ) } ========================= #
F %= num, lambda h, s: ConstantNumberNode(s[1]), None
F %= opar + E + cpar, lambda h, s: s[2], None, None, None
#                                                                         #
# ======================================================================= #
############################# END PRODUCTIONS #############################


Ensamblemos el pipeline de evaluación con los elementos que hemos ido implementando a lo largo de las pasadas clases.

Se realizará la siguiente cadena de transformaciones:
```
Entrada -> Tokens -> Parse Izquierdo -> AST -> Resultado
```    

In [49]:
from cmp.utils import Token
from cmp.languages import BasicHulk
from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
from cmp.tools.evaluation import evaluate_parse

hulk = BasicHulk(G)
firsts = hulk.firsts
follows = hulk.follows
tokenize_text = hulk.tokenizer

M = build_parsing_table(G, firsts, follows)
parser = metodo_predictivo_no_recursivo(G, M)

def run_pipeline(text, value, parser, formatter):
    tokens = tokenize_text(text)
    pprint(tokens, '================Tokens================')
    left_parse = parser(tokens)
    pprint(left_parse, '==============Left-Parse==============')
    ast = evaluate_parse(left_parse, tokens)
    pprint(formatter(ast), '=================AST==================')
    result = ast.evaluate()
    pprint(f'{text} = {result}', '================Result================')
    assert result == value

Comprobemos que la asociatividad de los operadores no se perdió.

In [50]:
run_pipeline('1 - 1 - 1', -1, parser, printer)

================Tokens================
[
   num: 1
   -: -
   num: 1
   -: -
   num: 1
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> num
   Y -> e
   X -> e
]
=================AST==================
\__<expr> MinusNode <expr>
	\__<expr> MinusNode <expr>
		\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 1
	\__ ConstantNumberNode: 1
================Result================
1 - 1 - 1 = -1.0


In [51]:
run_pipeline('1 - ( 1 - 1 )', 1, parser, printer)

================Tokens================
[
   num: 1
   -: -
   (: (
   num: 1
   -: -
   num: 1
   ): )
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> ( E )
   E -> T X
   T -> F Y
   F -> num
   Y -> e
   X -> - T X
   T -> F Y
   F -> num
   Y -> e
   X -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> MinusNode <expr>
	\__ ConstantNumberNode: 1
	\__<expr> MinusNode <expr>
		\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 1
================Result================
1 - ( 1 - 1 ) = 1.0


## Adicionando operador _potencia_

Añadamos el operador potencia al lenguaje. Para ello, realizaremos las modificaciones pertinentes a cada una de las fases del evaluador. No será necesario **copia y pegar** código de otras clases, ni modificar el código fuente del módulo `cmp` que se distribuye junto al _notebook_.

Usaremos el símbolo `^` para denotar al operador potencia. Este es un operador binario que computa $a^b$ siendo, `a` y `b` los operandos izquierdo y derecho respectivamente. Por ejemplo, `2 ^ 4` computa $2^4$. El operador potencia asocia hacia la derecha (contrario a los operadores: +, -, \* y /). Por tanto, `4 ^ 3 ^ 2` computa $4^{3^2} = 4^9$ en lugar de $(4^3)^2 = 4^6$.

In [52]:
class UnitaryNode(Node):
    def __init__(self, operator):
        self.operator = operator

    def evaluate(self):
        op = self.operator
        return self.operate(op)

    @staticmethod
    def operate(operator):
        raise NotImplementedError()
        
class PowerNode(BinaryNode):
    @staticmethod
    def operate(lvalue, rvalue):
        return lvalue ** rvalue

In [53]:
G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y, P, Z = G.NonTerminals('T F X Y P Z')
plus, minus, star, div, opar, cpar, num, power = G.Terminals('+ - * / ( ) num ^')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
X %= plus + T + X, lambda h, s: s[3], None, None, lambda h,s: PlusNode(h[0], s[2])
X %= minus + T + X, lambda h, s: s[3], None, None, lambda h,s: MinusNode(h[0], s[2])
X %= G.Epsilon, lambda h, s: h[0]
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
T %= F + Y, lambda h, s: s[2], None, lambda h, s: s[1]
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
Y %= star + F + Y, lambda h, s: s[3], None, None, lambda h, s: StarNode(h[0], s[2])
Y %= div + F + Y, lambda h, s: s[3], None, None, lambda h, s: DivNode(h[0], s[2])
Y %= G.Epsilon, lambda h, s: h[0]
#                                                                         #
# ============================ { F --> P Z } ============================ #
#
F %= P + Z, lambda h, s: s[2], None, lambda h, s: s[1]
#
# ======================= { Z --> ^ F | epsilon } =================================#
#
Z %= power + F, lambda h, s: PowerNode(h[0], s[2]), None, None
Z %= G.Epsilon, lambda h, s: h[0]
#
# ======================= { P --> num | ( E ) } ========================= #
P %= num, lambda h, s: ConstantNumberNode(s[1]), None
P %= opar + E + cpar, lambda h, s: s[2], None, None, None
#                                                                         #
# ======================================================================= #
############################# END PRODUCTIONS #############################

fixed_tokens = { lex: Token(lex, G[lex]) for lex in '+ - * / ( ) ^'.split() }

def tokenize_text(text):
        tokens = []
        for item in text.split():
            try:
                float(item)
                token = Token(item, G['num'])
            except ValueError:
                token = fixed_tokens[item]
            tokens.append(token)
        eof = Token('$', G.EOF)
        tokens.append(eof)
        return tokens

Como ayuda se proveen los conjunto _First_ y _Follow_ precomputados de lo que consideramos la gramática **natural** a obtener. De igual forma se tiene la tabla y parser LL(1). Siéntase libre de utilizar el código siguiente pero puede reemplazarlo por sus propias implementaciones en caso de que no pueda (o no entienda) cómo utilizarlo.

In [54]:
from cmp.utils import ContainerSet

# Computes First(alpha), given First(Vt) and First(Vn)
# alpha in (Vt U Vn)*
def compute_local_first(firsts, alpha):
    first_alpha = ContainerSet()
    
    try:
        alpha_is_epsilon = alpha.IsEpsilon
    except:
        alpha_is_epsilon = False
    
    ###################################################
    # alpha == epsilon ? First(alpha) = { epsilon }
    ###################################################
    
    if alpha_is_epsilon:
        first_alpha.set_epsilon()
    
    # Para saber si epsilon pertenece a todos los Xi
    all_epsilon = True
    for xi in alpha:
        # Encuentra el primer First(Xi) al que epsilon no pertenezca
        if not firsts[xi].contains_epsilon: 
            # Se agregan todos los simbolos que no sean epsilon
            for symbol in firsts[xi]:
                if not symbol.IsEpsilon:
                    first_alpha.add(symbol)
            all_epsilon = False
            break
        
    # Si epsilon pertenece a todos los First(Xi) entonces epsilon pertenece 
    # a First(alpha)
    if all_epsilon:
        first_alpha.set_epsilon()
    
    # First(alpha)
    return first_alpha

# Computes First(Vt) U First(Vn) U First(alpha)
# P: X -> alpha
def compute_firsts(G):
    firsts = {}
    change = True
    
    # init First(Vt)
    for terminal in G.terminals:
        firsts[terminal] = ContainerSet(terminal)
        
    # init First(Vn)
    for nonterminal in G.nonTerminals:
        firsts[nonterminal] = ContainerSet()
    
    while change:
        change = False
        
        # P: X -> alpha
        for production in G.Productions:
            X = production.Left
            alpha = production.Right
            
            # get current First(X)
            first_X = firsts[X]
                
            # init First(alpha)
            try:
                first_alpha = firsts[alpha]
            except KeyError:
                first_alpha = firsts[alpha] = ContainerSet()
            
            # CurrentFirst(alpha)???
            local_first = compute_local_first(firsts, alpha)
            
            # update First(X) and First(alpha) from CurrentFirst(alpha)
            change |= first_alpha.hard_update(local_first)
            change |= first_X.hard_update(local_first)
                    
    # First(Vt) + First(Vt) + First(RightSides)
    return firsts

from itertools import islice

def compute_follows(G, firsts):
    follows = { }
    change = True
    
    local_firsts = {}
    
    # init Follow(Vn)
    for nonterminal in G.nonTerminals:
        follows[nonterminal] = ContainerSet()
    follows[G.startSymbol] = ContainerSet(G.EOF)
    
    while change:
        change = False
        
        # P: X -> alpha
        for production in G.Productions:
            X = production.Left
            alpha = production.Right
            
            follow_X = follows[X]
            
            # Se recorre la parte derecha de la produccion,
            # como zeta no interesa, tomamos a beta como
            # la posicion de Y + 1, luego tanto si
            # first[beta].contains_epsilon o beta es None
            # se aplica Follow(X) subset of Follow(Y)
            # sino First(beta) - {epsilon} subset Follow(Y)
            for i in range(len(alpha)):
                try:
                    beta = alpha[i + 1]
                except IndexError:
                    beta = None
                for j in range(i + 2, len(alpha)):
                    beta += alpha[j]
                if alpha[i].IsNonTerminal:
                    if beta != None:
                        change |= follows[alpha[i]].update(firsts[beta])
                    if beta == None or firsts[beta].contains_epsilon:
                        change |= follows[alpha[i]].update(follow_X)

    # Follow(Vn)
    return follows


In [55]:
from cmp.languages import PowHulk
from cmp.utils import ContainerSet
from firsts_follows import compute_firsts, compute_follows

firsts = compute_firsts(G)
follows = compute_follows(G, firsts)

from cmp.tools.parsing import build_parsing_table, metodo_predictivo_no_recursivo
M = build_parsing_table(G, firsts, follows)
parser = metodo_predictivo_no_recursivo(G, M)

================Tokens================
[
   num: 4
   ^: ^
   num: 3
   ^: ^
   num: 2
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> P Z
   P -> num
   Z -> ^ F
   F -> P Z
   P -> num
   Z -> ^ F
   F -> P Z
   P -> num
   Z -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> PowerNode <expr>
	\__ ConstantNumberNode: 4
	\__<expr> PowerNode <expr>
		\__ ConstantNumberNode: 3
		\__ ConstantNumberNode: 2
================Result================
4 ^ 3 ^ 2 = 262144.0


**Comprobemos que el operador potencia asocia hacia la derecha.**

In [56]:
run_pipeline('4 ^ 3 ^ 2', 262144, parser, printer)

================Tokens================
[
   num: 4
   ^: ^
   num: 3
   ^: ^
   num: 2
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> P Z
   P -> num
   Z -> ^ F
   F -> P Z
   P -> num
   Z -> ^ F
   F -> P Z
   P -> num
   Z -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> PowerNode <expr>
	\__ ConstantNumberNode: 4
	\__<expr> PowerNode <expr>
		\__ ConstantNumberNode: 3
		\__ ConstantNumberNode: 2
================Result================
4 ^ 3 ^ 2 = 262144.0


**Comprobemos que tiene más precedencia que el resto de los operadores.**

In [57]:
run_pipeline('2 * 3 ^ 4 + 1 * 5', 167, parser, printer)

================Tokens================
[
   num: 2
   *: *
   num: 3
   ^: ^
   num: 4
   +: +
   num: 1
   *: *
   num: 5
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> P Z
   P -> num
   Z -> e
   Y -> * F Y
   F -> P Z
   P -> num
   Z -> ^ F
   F -> P Z
   P -> num
   Z -> e
   Y -> e
   X -> + T X
   T -> F Y
   F -> P Z
   P -> num
   Z -> e
   Y -> * F Y
   F -> P Z
   P -> num
   Z -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> PlusNode <expr>
	\__<expr> StarNode <expr>
		\__ ConstantNumberNode: 2
		\__<expr> PowerNode <expr>
			\__ ConstantNumberNode: 3
			\__ ConstantNumberNode: 4
	\__<expr> StarNode <expr>
		\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 5
================Result================
2 * 3 ^ 4 + 1 * 5 = 167.0


**Comprobemos que puede subordinarse a otros operadores usando paréntesis.**

In [58]:
run_pipeline('3 ^ ( 1 + 1 ) ^ 2', 81, parser, printer)

================Tokens================
[
   num: 3
   ^: ^
   (: (
   num: 1
   +: +
   num: 1
   ): )
   ^: ^
   num: 2
   $: $
]
==============Left-Parse==============
[
   E -> T X
   T -> F Y
   F -> P Z
   P -> num
   Z -> ^ F
   F -> P Z
   P -> ( E )
   E -> T X
   T -> F Y
   F -> P Z
   P -> num
   Z -> e
   Y -> e
   X -> + T X
   T -> F Y
   F -> P Z
   P -> num
   Z -> e
   Y -> e
   X -> e
   Z -> ^ F
   F -> P Z
   P -> num
   Z -> e
   Y -> e
   X -> e
]
=================AST==================
\__<expr> PowerNode <expr>
	\__ ConstantNumberNode: 3
	\__<expr> PowerNode <expr>
		\__<expr> PlusNode <expr>
			\__ ConstantNumberNode: 1
			\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 2
================Result================
3 ^ ( 1 + 1 ) ^ 2 = 81.0


## Adicionando _declaraciones de variables_

Añadamos variables al lenguaje de expresiones para acercarnos más a `HULK`. Para ello, agregue la expresión `let-in` al lenguaje. Dicha expresión sigue la siguiente sintaxis:

```
let
    <declaration-list>
in
    <expr>
```
donde `<expr>` denota cualquier expresión del lenguaje (incluyendo el propio `let-in`) y `<declaration-list>` representa una secuencia de declaraciones de la forma `<id> = <expr>` separadas por "`,`".

El valor de evaluación de la expresión `let-in` es el valor de evaluación de `<expr>`.  
Las variables declaradas en `<declaration-list>` serán visibles a partir de su declaración pero únicamente dentro de la expresión `let-in` que las contiene (incluye `<expr>`). Si `<expr>` contiene a su vez una expresión `let-in`, la declaración de una variable con el mismo nombre que una en el `let-in` padre **ocultará** la del padre.  
Por ejemplo, la expresión:
```
let
    x = 1,
    y = 2
in
    3 + (let x = 4, z = 5 in x + y + z) + x
```
equivalente a:
```
let x = 1, y = 2 in 3 + (let x = 4, z = 5 in x + y + z) + x
```
evalúa `15`.

In [ ]:
declaration_list = G.nonTerminals('declaration_list')
let, inn, id, equal, comma = G.Terminals('let in id = ,')

E %= let + declaration_list + inn + E, None, None, None, None
E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]
declaration_list %= id + equal + E, None, None, None, None
declaration_list %= id + equal + E + comma + declaration_list, None, None, None, None

In [ ]:
# An empty cell. Just in case you don't know how to create new ones

In [ ]:
# An empty cell. Just in case you don't know how to create new ones